In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

In [ ]:
# Assume that preprocessed_data is the pre-processed dataset that has been prepared
# according to the instructions in the previous steps.

# Define the number of qubits required to encode the data.
num_qubits = len(preprocessed_data[0])

# Initialize a quantum register with the required number of qubits.
qr = QuantumRegister(num_qubits)

# Create a quantum circuit with the initialized quantum register.
circ = QuantumCircuit(qr)

# Loop through the pre-processed data and apply amplitude encoding to each qubit.
for i, sample in enumerate(preprocessed_data):
    for j, value in enumerate(sample):
        circ.ry(2 * value, qr[j])

# Save the encoded quantum states to be used in further processing.
encoded_states = circ

In [ ]:
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.feature_maps import SecondOrderExpansion
from qiskit.aqua.components.multiclass_extensions import AllPairs
from qiskit.aqua.utils import split_dataset_to_data_and_labels

# Assume that encoded_states is the encoded quantum states that have been prepared
# according to the instructions in the previous steps.

# Split the encoded states into training and test sets.
training_data, test_data, training_labels, test_labels = split_dataset_to_data_and_labels(encoded_states, labels)

# Define the number of qubits required to run the QSVM algorithm.
num_qubits = len(encoded_states.qubits)

# Initialize a feature map for the QSVM algorithm.
feature_map = SecondOrderExpansion(num_qubits)

# Initialize an all-pairs multiclass extension for the QSVM algorithm.
multiclass_extension = AllPairs(feature_map)

# Initialize a QSVM algorithm instance with the feature map and multiclass extension.
qsvm = QSVM(feature_map, training_data, training_labels, multiclass_extension)

# Initialize a quantum instance to run the QSVM algorithm on a quantum computing platform.
quantum_instance = QuantumInstance(backend, shots=num_shots)

# Run the QSVM algorithm on the quantum instance and obtain the results.
results = qsvm.run(quantum_instance)

# Save the results obtained from the quantum computing platform for further analysis and comparison with the classical deep learning results.


In [ ]:
# Load test data
test_data = pd.read_csv('KDDTest+.txt', header=None, names=col_names)

# Preprocess test data
test_data = preprocess_data(test_data)

# Separate features and labels
test_labels = test_data.pop('label')
test_features = test_data.values

# Convert to one-hot encoding
test_features = encoder.transform(test_features).toarray()

# Normalize features
test_features = scaler.transform(test_features)


In [ ]:
# Encode test features using quantum encoding
test_encoded = []
for features in test_features:
    encoded_features = quantum_encode(features, qubits)
    test_encoded.append(encoded_features)
test_encoded = np.array(test_encoded)


In [ ]:
# Predict using quantum model
quantum_predictions = quantum_model.predict(test_encoded)


In [ ]:
# Evaluate quantum model
quantum_accuracy = accuracy_score(test_labels, quantum_predictions)
quantum_precision = precision_score(test_labels, quantum_predictions, average='weighted')
quantum_recall = recall_score(test_labels, quantum_predictions, average='weighted')
quantum_f1 = f1_score(test_labels, quantum_predictions, average='weighted')

print(f'Quantum accuracy: {quantum_accuracy:.4f}')
print(f'Quantum precision: {quantum_precision:.4f}')
print(f'Quantum recall: {quantum_recall:.4f}')
print(f'Quantum F1 score: {quantum_f1:.4f}')
